<a href="https://colab.research.google.com/github/t1nh233/predict_vnindex_stacked_lstm/blob/main/lstm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing ins

In [10]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

In [2]:
## LOAD VA CLEAN DU LIEU DAU VAO

# Doc du lieu
url = "https://raw.githubusercontent.com/t1nh233/predict_vnindex_stacked_lstm/refs/heads/main/vn_index_historical_data_9_12.csv"
vnindex = pd.read_csv(url)
vnindex.head()

# # Xu ly du lieu thoi gian va dua Date thanh index
vnindex['Date'] = pd.to_datetime(vnindex['Date'], format='%m/%d/%Y')
vnindex = vnindex.sort_values(by='Date')
vnindex.set_index('Date', inplace=True)

# Chuyen doi ve dung dinh dang va dung don vi
columns_convert_to_float = ["Price", "Open", "High", "Low"]
for column in columns_convert_to_float:
    vnindex[column] = vnindex[column].str.replace(",", "").astype(float)

# Ham ho tro chuyen doi Volume va Change
def convert_volume(vol_str):
    multiplier_dict = {"K": 1000, "M": 1000000, "B": 1000000000}

    if not isinstance(vol_str, str):
        return vol_str

    vol_str = vol_str.upper().strip()
    last_char = vol_str[-1]

    if last_char in multiplier_dict.keys():
        multiplier_val = multiplier_dict[last_char]
        number_part = vol_str[:-1]
    else:
        multiplier_val = 1
        number_part = vol_str

    try:
        number = float(number_part.replace(',', ''))
    except ValueError:
        return np.nan

    return number * multiplier_val

def convert_change(change_str):
    if not isinstance(change_str, str):
        return change_str

    if change_str.endswith('%'):
        number_part = change_str[:-1]
    else:
        number_part = change_str

    try:
        number = float(number_part.replace(',', ''))
    except ValueError:
        return np.nan

    return number / 100.0


# Bien doi Volume va Change
vnindex['Vol.'] = vnindex['Vol.'].apply(convert_volume)
vnindex['Change %'] = vnindex['Change %'].apply(convert_change)

vnindex.rename(columns={'Price': 'Close', 'Vol.': 'Volume'}, inplace=True)

print(vnindex.info())
print(vnindex.head())

## DONE PHAN LOAD VA CLEAN DU LIEU DAU VAO

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3743 entries, 2010-12-09 to 2025-12-09
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Close     3743 non-null   float64
 1   Open      3743 non-null   float64
 2   High      3743 non-null   float64
 3   Low       3743 non-null   float64
 4   Volume    3743 non-null   float64
 5   Change %  3743 non-null   float64
dtypes: float64(6)
memory usage: 204.7 KB
None
             Close    Open    High     Low    Volume  Change %
Date                                                          
2010-12-09  460.45  450.59  460.79  450.16   60570.0    0.0168
2010-12-10  473.06  464.81  473.06  462.39   94110.0    0.0274
2010-12-13  490.22  485.82  490.30  485.73   81980.0    0.0363
2010-12-14  489.65  493.72  496.42  480.64  134950.0   -0.0012
2010-12-15  493.47  492.80  497.09  491.51   95680.0    0.0078


In [12]:
## CHUAN BI DU LIEU CHO MO HINH

# Them cot label vao cuoi cua moi ngay
vnindex['label'] = vnindex['Close'].shift(-1)

## Tinh toan cac chi so nang cao
# RSI (14)
vnindex['RSI'] = ta.rsi(vnindex['Close'], length=14)

# MACD(12, 26, 9)
vnindex['MACD'] = ta.macd(vnindex['Close'])['MACD_12_26_9']
vnindex['MACD_Signal'] = ta.macd(vnindex['Close'])['MACDs_12_26_9']
vnindex['MACD_Hist'] = ta.macd(vnindex['Close'])['MACDh_12_26_9']

# SMA(20)
vnindex['SMA'] = ta.sma(vnindex['Close'], length=20)

# EMA(50)
vnindex['EMA'] = ta.ema(vnindex['Close'], length=50)

# BB(20, 2.0)
bb_values = ta.bbands(vnindex['Close'], length=20, std=2.0)
vnindex['BB_Width'] = ta.bbands(vnindex['Close'], length=20, std=2.0)['BBB_20_2.0_2.0']
vnindex['BB_Percentage'] = ta.bbands(vnindex['Close'], length=20, std=2.0)['BBP_20_2.0_2.0']

# Xoa cac dong NaN(co it nhat 1 gia tri Nan trong hang = xoa)
vnindex_clean = vnindex.dropna()

# print(vnindex_clean.head())
# print(vnindex_clean.info())

## Chon va sap xep lai cac features

feature_columns = ['Close', 'Volume', 'RSI', 'SMA', 'MACD_Hist', 'BB_Width', 'BB_Percentage']

target_column = 'label'
all_cols = vnindex_clean.columns.tolist()
target_index = all_cols.index(target_column)

train_columns = feature_columns + [target_column]
train_data = vnindex_clean[train_columns].copy()

train_all_cols = train_data.columns.tolist()
target_index_after = train_all_cols.index(target_column)

# print(train_data.head())
# print(train_data.info())

# print(feature_columns)
# print(target_index)
# print(target_index_after)

## Scale ve (0,1) sau do gom thanh cac window_lag co size=30

def create_windows(data, window_size, target_col_index):
    X, y = [], []

    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size, :-1])
        y.append(data[i + window_size, target_col_index])

    return torch.tensor(np.array(X), dtype=torch.float32), torch.tensor(np.array(y), dtype=torch.float32).unsqueeze(1)

WINDOW_SIZE = 30
TARGET_INDEX = target_index_after

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(train_data)

X_train, y_train = create_windows(scaled_data, WINDOW_SIZE, TARGET_INDEX)

print(X_train.shape)
print(y_train.shape)

torch.Size([3663, 30, 7])
torch.Size([3663, 1])
